# Introduction to Biopython 2

### Key concepts

 - **SeqRecord object:** The `SeqRecord` object used in _Biopython_ to hold a sequence, as a `Seq` object, with identifiers `id` and `name`, `description` and optionally `annotation` and other sub-features. 
  <details>
    <summary>
        <span style="color: purple">Click here to display more information</span>
    </summary>
    <p>The <code>SeqRecord</code> object used in <em>Biopython</em> to hold a sequence, as a <a href="#Seq"><code>Seq</code></a> object, with identifiers <code>id</code> and <code>name</code>, description and optionally annotation and sub-features. The following table contains the <code>SeqRecord</code> attributes and the information they hold.</p>

    <blockquote>
            <table>
                <tr>
                   <td>.seq</td>
                   <td>Seq object containing a sequence</td>
                </tr>
                <tr>
                   <td>.id</td>
                   <td>Primary ID used to identify the sequence in a string format</td>
                </tr>
                <tr>
                   <td>.name</td>
                   <td> In some cases this will be the same as the accession number, but it could also be a clone name in a string a string format</td>
                </tr>
                <tr>
                   <td>.description</td>
                   <td>Brief description or expressive name for the sequence</td>
                </tr>
                <tr>
                   <td>.letter_annotations</td>
                   <td>
                   Dictionary of additional information about the letters in the sequence.
                   The keys are the name of the information (e.g. "phread_quality") and the value (as a list, tuple, string,...) has the same length as the
                   sequence itself (e.g. [40, 40, 38, 30, ...]).
                   </td>
                </tr>
                <tr>
                   <td>.annotations</td>
                   <td>
                    A dictionary of additional information about the sequence. The keys
                    are the name of the information, and the information is contained in
                    the value.
                   </td>
                </tr>
                 <tr>
                   <td>.features</td>
                   <td>
                   A list of SeqFeature objects with more structured information about the
                   features on a sequence (e.g. position of genes on a genome, or domains
                   on a protein sequence). See more on section 4.3 of the [documentation][docu].
                   </td>
                </tr>
                <tr>
                   <td>.dbxrefs</td>
                   <td>A list of database cross-references as strings (e.g. ['Project:58037']).</td>
                </tr>
             </table></blockquote>
    
    We will mainly be using the first 4 attributes. For example, the `example1.fa` file contains only two lines:
                
    >example1 this is a simple example<br>
    GATTACA-A
    
</details>



 - **Seq object:** The Seq attribute in the `SeqRecord` object is the minimum information needed to create an instance of this class. It consist on a sequence in the form of a `Python` `string` which offers many of the same methods along with additional ones. 
<details>
    <summary><span style="color: purple">Click here to display more information</span></summary>
    <p>The <code>Seq</code> attribute is the minimum information needed to create an instance of a <code>SeqRecord</code>. Like <code>SeqRecord</code>, the <code>Seq</code> object has its own set of attributes and its own module <code>Bio.Seq</code> which can be imported using <code>from Bio.Seq import Seq</code>.</p>
    <p>You need to keep in mind that like <code>Python</code> strings, <code>Seq</code> objects do not support item assignments and in order to modify them, we need to transform them into a <code>MutableSeq</code> object. To do this we need to import it using <code>from Bio.Seq import MutableSeq</code> and transform the <code>Seq</code> sequence through reassignment (e.g. <code>sequence = MutableSeq(sequence)</code>).</p>

</details>


 
Both of these objects are available in  their own module and can be imported using `from Bio.SeqRecord import SeqRecord` and `from Bio.Seq import Seq`.


----

### 2. Importing local files


If we wanted to manually import a sequence fasta file using `Python`, we would have to manually define the `ID/name` and read the rest of the file and store it as a sequence. The resulting function would look something like the following. 

In [1]:
## Read fasta file python function:
def read_fasta(fastafile):
    
    with open(fastafile) as f_inp:
        lines = f_inp.readlines()
    
    seqs = [Seq("")]
    seqs[0].id = lines[0][1:].split()[0].strip() # 1st word after the ">"
    seqs[0].description = " ".join(lines[0][1:].split()[1:]) # rest of the line after id ">"
    seq = [] 
    i = 0
    for line in lines[1:]:
        if line.startswith('>'): #new sequence
            seqs[i].seq = ''.join(seq)
            seqs.append(Seq(""))
            i += 1
            seq = []
            seqs[i].id = lines[0][1:].split()[0].strip()
            seqs[i].description = " ".join(lines[0][1:].split()[1:])
        else:
            seq.append(line.strip()) # keep inputing sequence
    seqs[0].seq = ''.join(seq)
    return seqs

# Example
from Bio.Seq import Seq
seq = read_fasta('example1.fa')[0]
print(seq.id)
print(seq.description)
print(seq.seq)

example0
this is a simple example
GATTACA-A


Then we would have to construct a specific function for each of the file formats. The `Bio.SeqIO` module allows a straightforward and uniform way for reading and writing sequence files in different formats, processing them as `SeqRecord` objects (view _Introduction to key concepts_ section).  The main function in `SeqIO` is **`Bio.SeqIO.parse()`**:
- **Input**: a **file's path** and **its format's name**.
- **Output**: an object or an iterator with objects of the `SeqRecord` class. 

In [2]:
from Bio import SeqIO

record = SeqIO.parse("example1.fa", "fasta") # get record as SeqRecord

[print(att) for att in list(record)] # print all information

ID: example0
Name: example0
Description: example0 this is a simple example
Number of features: 0
Seq('GATTACA-A')


[None]

1. Files containing **a single record** can also be read using the function **`SeqIO.read()`** which takes same arguments as `SeqIO.parse()`. If there is more than one record in the file, an exception is raised.

In [3]:
record = SeqIO.read("example1.fa", "fasta")
print(record)

ID: example0
Name: example0
Description: example0 this is a simple example
Number of features: 0
Seq('GATTACA-A')


 2. If the file contains **several sequences**, we can store them in a list the following way.

In [4]:
records = list(SeqIO.parse("example2.fa", "fasta"))

# to access the 1st SeqRecord in records:
print(records[0])

ID: example1
Name: example1
Description: example1 this is simple example 1
Number of features: 0
Seq('THIS-IS-SEQUENCE-4')


In [5]:
# to access all of the sequences of the document:
for rec in records:
    print(rec.seq, end= "\n\n")

THIS-IS-SEQUENCE-4

THIS-IS-SEQUENCE-1

THIS-IS-SEQUENCE-3

SOMETHING-IS-WRONG

THIS-IS-SEQUENCE-2

THIS-IS-SEQUENCE-1

THIS-IS-SEQUENCE-6



### 3.  Modifying data

In the previous section, we demonstrated how to extract data from a `SeqRecord`. Once we have obtain the record, we can alter this data. 

1. The attributes of a SeqRecord can be modified directly:
```python
from Bio import SeqIO
record = SeqIO.parse("example1.fa", "fasta") 
record.id = "Gattaca"
```




In [6]:
print("current Name is:", record.name, end = "\n\n")
record.name = "Gattaca" #change name
print("changed Name is:", record.name, end = "\n\n")

#print(record) # check all the attribute's content
record.name = "example0" #change name back

current Name is: example0

changed Name is: Gattaca



In [7]:
# check what happens we try to alter the sequence's content
record.seq[3] = "A" 

TypeError: 'Seq' object does not support item assignment

2. Transform them into a `MutableSeq`object.

In order to be able to only modify a specific position of a sequence we could turn them into a Python `list` then alter the postion and then use `.join()`to reassign the `.seq` content of record:


In [8]:
recordlist = list(record.seq)
print(recordlist) # original
recordlist[-2] = "A"
print(recordlist, end="\n\n") # modified

record.seq = "".join(recordlist)
print(record)

['G', 'A', 'T', 'T', 'A', 'C', 'A', '-', 'A']
['G', 'A', 'T', 'T', 'A', 'C', 'A', 'A', 'A']

ID: example0
Name: example0
Description: example0 this is a simple example
Number of features: 0
'GATTACAAA'


Alternatively, we can transform it into a `MutableSeq`object change it and use `.toseq()` when reassigning the variable to `record.seq`. To do this we need to import it
using `from Bio.Seq import MutableSeq`.

In [9]:
from Bio.Seq import MutableSeq

mut_seq = MutableSeq(record.seq) # get mutable version

mut_seq[-2] = "-" # modify the sequence

record.seq = mut_seq.toseq()# back to a Seq object and reasign variable

print(record) # check all the attribute's content

ID: example0
Name: example0
Description: example0 this is a simple example
Number of features: 0
Seq('GATTACA-A')


### <span style='color: blue'>Activity 2.  Modify a `Bio.Record` object</span>

1. Change the `.seq` sequence of the `SeqRecord` in the `record` variable containing the information in the file `example1.fa`  to be _"THIS-IS-SEQUENCE-0"_.   


In [10]:
#solution
record.seq = Seq("".join([...]))
print(record)

ID: example0
Name: example0
Description: example0 this is a simple example
Number of features: 0
Seq('THIS-IS-SEQUENCE-0')


2. Do the same for the sequence of As you have seen the `example2.fa` file contains 7 records. The sequences should follow the pattern _"THIS-IS-SEQUENCE-#"_ (where "#" is 1, 2 and 3 respectively). Now change the `.seq` of the records 4 to 7 to follow the same pattern.     


In [11]:
#[print(rec, end= "\n\n") for rec in records]

In [16]:
#solution

for rec in ...:
    rec.seq = Seq("".join([...]))

[print(rec, end= "\n"+"..."+"\n") for rec in records[0::3]]

ID: example1
Name: example1
Description: example1 this is simple example 1
Number of features: 0
Seq('THIS-IS-SEQUENCE-4')
...
ID: example4
Name: example4
Description: example4 this is simple example 4
Number of features: 0
Seq('THIS-IS-SEQUENCE-4')
...
ID: example7
Name: example7
Description: example7 this is simple example 7
Number of features: 0
Seq('THIS-IS-SEQUENCE-7')
...


[None, None, None]

------

## Exporting local files


You can use the `SeqIO.write()` function to output sequences files. This
function takes three arguments:
- A `SeqRecord` object or a list of them.
- A handle or filename to write to.
- The sequence format (e.g. _"fasta"_).

The following shows how this would work (see more on the
[activities][linkactivity]. In this case we are storing the list of records
_records_ into the output file _output_file.fa_ which will have a _fasta_ 
format.

```python
SeqIO.write(records, "output.fa" , "fasta")
```

This is the simple, straight-forward way, alternatively we can use 
`with open()`. The context manager (with statement) ensures that the file is properly closed after writing. 

```python
with open("output.fa", "w") as file_out:
    SeqIO.write(records, file_out, "fasta")
```

Both approaches achieve the same result, but using `with open()`
will allow you to perform additional operations. For example if the output file
isn't empty and your goal is to append the _records_ at the end of the file, 
you can easily achieve this by replacing the "w" in `open` by "a" (for append
mode). 


```python
with open("output.fa", "a") as file_out:
    SeqIO.write(records, file_out, "fasta")

```


### <span style='color: blue'>Activity 3. Exporting a file</span>

1. Using `SeqIO.write()` to create a new `fasta` file that contains the information of the modified `record` variable.  <span style='color: green'> Name the file *output_file.fa*</span>

In [13]:

SeqIO.write(...) # the output is the number records saved


1


2. Using `SeqIO.write()` to write at the end of the file, the content of the modified records of `example2.fa` stored in the `records` variable. Check your local directory to observe the resulting file 

In [14]:
with open(...) as file_out:
    SeqIO.write(...)

#### Extra

You can use existent files to create new files of different formats with the 
same content. You could read the file, using the `Bio.SeqIO.parse()` and pass
the `SeqRecord` iterator to the `Bio.SeqIO.write()` and save it using a 
different file format. However, `SeqIO` offers the `SeqIO.convert` function
which takes four arguments and does the this concisely. 

For example to turn a _GenBank_ file into a _Fasta_  file we can:

```python
from Bio import SeqIO
count = SeqIO.convert("output_file.fa", "fasta", "converted.txt", "tab")
```

